## 1. Importing relevant Python libraries for the assignment

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from urllib.request import urlopen # library for working with URLs

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                              \failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='0_73_pypy|1_73_pypy|2_73_pypy|4_73_pypy|3_73_pypy']
  - geopy -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']
  - rsa -> python[version='

grpcio -> protobuf[version='>=3.5.0']
ibm-wsrt-py37main-main -> protobuf==3.12.3[build=*]
tensorflow-estimator -> protobuf[version='>=3.6.1']
tensorboard -> protobuf[version='>=3.3.0|>=3.4.0|>=3.6.0|>=3.8.0|>=3.6.1']

Package jedi conflicts for:
ipython -> jedi[version='>=0.10|>=0.10,<0.18|>=0.16']
ibm-wsrt-py37main-main -> jedi==0.17.1[build=*]

Package retrying conflicts for:
plotly -> retrying[version='>=1.3.3']
ibm-wsrt-py37main-main -> retrying==1.3.3[build=*]

Package cryptography conflicts for:
pyjwt -> cryptography[version='>=1.0|>=1.4|>=2.6,<3.0.0']
oauthlib -> cryptography
pyopenssl -> cryptography[version='>=1.3|>=1.3.4|>=1.9|>=2.1.4|>=2.2.1|>=2.8|>=3.2']
ibm-wsrt-py37main-main -> cryptography==3.2.1[build=*]
secretstorage -> cryptography
urllib3 -> cryptography[version='>=1.3.4']

Package ipykernel conflicts for:
ibm-wsrt-py37main-main -> ipykernel==5.3.0[build=*]
ipywidgets -> ipykernel[version='>=4.2.2|>=4.5.1']
widgetsnbextension -> ipykernel[version='>=4.5.1']
notebook 

     |████████████████████████████████| 98 kB 9.5 MB/s  eta 0:00:01
     |████████████████████████████████| 94 kB 6.6 MB/s  eta 0:00:01
Libraries imported.


## 2. Web scrape and reformat Wikipedia file into Pandas df format

In [2]:
# extract the Wikipedia webpage as split lines for reading
url = "https://en.wikipedia.org/wiki/List_of_neighborhoods_of_Madrid"
response = requests.get(url)
lines = response.content.decode('utf-8').splitlines()
len_lines = len(lines)

# datarow variables
dummy = "" # redundant variable
district_name = ""
number = ""
neighborhood = ""
postal_code = ""
latitude_val = ""
longitude_val = ""
start_index = 0
end_index = 0

# define the dataframe columns
column_names = ['District', 'Number', 'Neighborhood', 'Postal Code', 'Latitude', 'Longitude'] 

# instantiate the dataframe
madrid_data = pd.DataFrame(columns=column_names)

# loop through every line
for i in range(0, len_lines):
    line = lines[i]

    # extract the header columns data into headers array and
    if "</td></tr></tbody></table>" in line:
        dummy = ""
        break
    # append the datarow variables as entries into data array
    elif "</td></tr>" in line:
        # Retrieve the latitude and the longitude coordinates of each neighborhood
        if line.find("</tbody></table>") < 0 and neighborhood != "":
            if neighborhood == "Los Cármenes":
                index = neighborhood.find("Los ") + len("Los ")
                neighborhood = neighborhood[index:len(neighborhood)]
            # for neighbourhoods without available coordinates, substitute them with Madrid
            elif neighborhood == "Valderrivas":
                neighborhood = "Madrid"
            elif neighborhood == "Casco Histórico de Barajas":
                neighborhood = "Madrid"

            # retrieve the postcode for each neighbourhood
            address = neighborhood + ' , Madrid, Spain'
            geolocator = Nominatim(user_agent="mc_explorer")
            location = geolocator.geocode(address)
            latitude_val = location.latitude
            longitude_val = location.longitude
            location = geolocator.reverse((latitude_val, longitude_val))
            postal_code = location.raw['address']['postcode']
            
            madrid_data = madrid_data.append({'District': district_name,
                                              'Number': number,
                                              'Neighborhood': neighborhood,
                                              'Postal Code': postal_code,
                                              'Latitude': latitude_val,
                                              'Longitude': longitude_val}, ignore_index=True)
    elif "<td rowspan=" in line:
        # extract district name
        if line.find(" title=") > -1:
            index = line.find(" title=") + len(" title=")
            rest = line[index:len(line)]
            if rest.find(">") > -1:
                index = rest.find(">") + len(">")
                rest = rest[index:len(rest)]
                if rest.find("<") > -1:
                    end_index = rest.find("<")
                    district_name = rest[0:end_index]
    elif "<td align=\"center\"><a href=" in line:
        # extract neighbourhood
        index = line.find("<td align=\"center\"><a href=") + len("<td align=\"center\"><a href=")
        rest = line[index:len(line)]
        if rest.find(" title=") > -1:
            index = rest.find(" title=") + len(" title=")
            rest = rest[index:len(line)]
            if rest.find(">") > -1:
                index = rest.find(">") + len(">")
                rest = rest[index:len(rest)]
                if rest.find("<") > -1:
                    end_index = rest.find("<")
                    neighborhood = rest[0:end_index]
    elif "<td align=\"center\">" in line:
        # extract district number
            if line.find("<td align=\"center\">") > -1 and line.find(" title=") < 0:
                start_index = line.find("<td align=\"center\">") + len("<td align=\"center\">")
                number = line[start_index:len(line)]
    else:
        dummy = ""

madrid_data

,District,Number,Neighborhood,Postal Code,Latitude,Longitude
0,Centro,11,Palacio,28001,40.415129,-3.715618
1,Centro,12,Embajadores,28012,40.409681,-3.701644
2,Centro,13,Cortes,28014,40.414348,-3.698525
3,Centro,14,Justicia,28004,40.423957,-3.695747
4,Centro,15,Universidad,28004,40.425310,-3.706630
...,...,...,...,...,...,...
125,San Blas-Canillejas,208,El Salvador,28001,40.444125,-3.627987
126,Barajas,211,Alameda de Osuna,28001,40.457581,-3.587975
127,Barajas,212,Aeropuerto,28001,40.494838,-3.574081
128,Barajas,213,Madrid,28013,40.416705,-3.703582


#### Use geopy library to get the latitude and longitude values of Madrid City.

In [3]:
address = 'Madrid, Spain'

geolocator = Nominatim(user_agent="mad_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid City are 40.4167047, -3.7035825.


## 2. Explore Neighborhoods in Madrid

#### Define Foursquare Credentials and Version

In [4]:
#CLIENT_ID = 'your-client-ID' # your Foursquare ID
#CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
CLIENT_ID = '4LJS4VEZUOLAZ350STJAD43WKIVBFXYONC22AVB0UQI3GGOM' # your Foursquare ID
CLIENT_SECRET = '0JP1QTFXCWXHJCJR4AXLBBIE2DILYAFDOLFUQ0CA1PRQNKJM' # your Foursquare Secret
ACCESS_TOKEN = 'SXDX1VFQBLANOQFXFOSWE2AC5FNXN3UUSZEL3DE41CJ3J2OJ' # your FourSquare Access Token
#VERSION = '20180605' # Foursquare API version
VERSION = '20180628' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on medical centers in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Italian restaurant' category, as we need info on Italian restaurants in the neighborhood.

In [ ]:
# Category IDs corresponding to Medical Centers were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

medical_category = '4bf58dd8d48988d104941735' # 'Root' category for all medical-related venues
doctors_office_venue = '4bf58dd8d48988d177941735'

medical_venue_categories = ['52e81612bcbc57f1066b7a3b','52e81612bcbc57f1066b7a3c','52e81612bcbc57f1066b7a3a',
                            '4bf58dd8d48988d178941735','4bf58dd8d48988d177941735','4bf58dd8d48988d194941735',
                            '522e32fae4b09b556e370f19','4bf58dd8d48988d196941735','58daa1558bbb0b01f18ec1f7',
                            '56aa371be4b08b9a8d5734ff','4f4531b14b9074f6e4fb0103','52e81612bcbc57f1066b7a39',
                            '58daa1558bbb0b01f18ec1d0','5744ccdfe4b0c0459246b4af','56aa371be4b08b9a8d57351d',
                            '56aa371be4b08b9a8d573526','4d954af4a243a5684765b473']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [ ]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found italian restaurants

#import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    italian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_italian = is_restaurant(venue_categories, specific_filter=italian_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_italian, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_italian:
                    italian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, italian_restaurants, location_restaurants

"""
# Try to load from local file system in case we did this before
restaurants = {}
italian_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('italian_restaurants_350.pkl', 'rb') as f:
        italian_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    #restaurants, italian_restaurants, location_restaurants = get_restaurants(latitudes, longitudes) # latitudes and longtitudes are Madrid neighbourhoods-based
"""

# Latitudes and Longtitudes are Madrid neighbourhoods'
restaurants, italian_restaurants, location_restaurants = get_restaurants(madrid_data['Latitude'], madrid_data['Longitude'])

"""
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('italian_restaurants_350.pkl', 'wb') as f:
        pickle.dump(italian_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
"""


In [ ]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Italian restaurants:', len(italian_restaurants))
print('Percentage of Italian restaurants: {:.2f}%'.format(len(italian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

In [ ]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

#### Let's create a function to repeat the same process to all the neighborhoods in Madrid

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        #https://api.foursquare.com/v2/venues/search?ll=29.253219,-81.732377&categoryId=4bf58dd8d48988d196941735&
        #    client_id=MYCLIENTID&client_secret=MYCLIENTSECRET&limit=1&v=20180628
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
#Now write the code to run the above function on each neighborhood and create a new dataframe called madrid_venues.
madrid_venues = getNearbyVenues(names=madrid_data['Neighborhood'],
                                latitudes=madrid_data['Latitude'],
                                longitudes=madrid_data['Longitude']
                                )

Palacio
Embajadores
Cortes
Justicia
Universidad
Sol
Imperial
Acacias
Chopera
Legazpi
Delicias
Palos de Moguer
Atocha
Pacífico
Adelfas
Estrella
Ibiza
Los Jerónimos
Niño Jesús
Recoletos
Goya
Fuente del Berro
Guindalera
Lista
Castellana
El Viso
Prosperidad
Ciudad Jardín
Hispanoamérica
Nueva España
Castilla
Bellas Vistas
Cuatro Caminos
Castillejos
Almenara
Valdeacederas
Berruguete
Gaztambide
Arapiles
Trafalgar
Almagro
Ríos Rosas
Vallehermoso
El Pardo
Fuentelarreina
Peñagrande
Pilar
La Paz
Valverde
Mirasierra
El Goloso
Casa de Campo
Argüelles
Ciudad Universitaria
Valdezarza
Valdemarín
El Plantío
Aravaca
Cármenes
Puerta del Ángel
Lucero
Aluche
Campamento
Cuatro Vientos
Las Águilas
Comillas
Opañel
San Isidro
Vista Alegre
Puerta Bonita
Buenavista
Abrantes
Orcasitas
Orcasur
San Fermín
Almendrales
Moscardó
Zofío
Pradolongo
Entrevías
San Diego
Palomeras Bajas
Palomeras Sureste
Portazgo
Numancia
Pavones
Horcajo
Marroquina
Media Legua
Fontarrón
Vinateros
Ventas
Pueblo Nuevo
Quintana
Concepción
San 

In [7]:
print(madrid_venues.shape)
madrid_venues.head()

(3928, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Palacio,40.415129,-3.715618,Santa Iglesia Catedral de Santa María la Real ...,40.415767,-3.714516,Church
1,Palacio,40.415129,-3.715618,Plaza de La Almudena,40.416320,-3.713777,Plaza
2,Palacio,40.415129,-3.715618,Cervecería La Mayor,40.415218,-3.712194,Beer Bar
3,Palacio,40.415129,-3.715618,Palacio Real de Madrid,40.417940,-3.714259,Palace
4,Palacio,40.415129,-3.715618,Taberna Rayuela,40.413179,-3.713496,Tapas Restaurant


In [8]:
madrid_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Villaverde Alto,10,10,10,10,10,10
Abrantes,9,9,9,9,9,9
Acacias,42,42,42,42,42,42
Adelfas,37,37,37,37,37,37
Aeropuerto,1,1,1,1,1,1
...,...,...,...,...,...,...
Valverde,4,4,4,4,4,4
Ventas,28,28,28,28,28,28
Vinateros,9,9,9,9,9,9


In [9]:
print('There are {} uniques categories.'.format(len(madrid_venues['Venue Category'].unique())))

There are 269 uniques categories.


## 3. Analyze Each Neighborhood

In [11]:
# one hot encoding
madrid_onehot = pd.get_dummies(madrid_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
madrid_onehot['Neighborhood'] = madrid_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [madrid_onehot.columns[-1]] + list(madrid_onehot.columns[:-1])
madrid_onehot = madrid_onehot[fixed_columns]

madrid_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,...,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Winery,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
madrid_onehot.shape

(3928, 269)

In [13]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [14]:
madrid_grouped = madrid_onehot.groupby('Neighborhood').mean().reset_index()
madrid_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,...,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Winery,Women's Store
0,Villaverde Alto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
1,Abrantes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
2,Acacias,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02381,0.0,0.0
3,Adelfas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
4,Aeropuerto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,Valverde,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
123,Ventas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
124,Vinateros,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
125,Vista Alegre,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0


In [15]:
madrid_grouped.shape

(127, 269)

In [16]:
#Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in madrid_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = madrid_grouped[madrid_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Villaverde Alto----
               venue  freq
0      Train Station   0.3
1         Restaurant   0.2
2              Diner   0.1
3  Mobile Phone Shop   0.1
4      Metro Station   0.1


----Abrantes----
                  venue  freq
0          Soccer Field  0.11
1           Pizza Place  0.11
2         Metro Station  0.11
3  Fast Food Restaurant  0.11
4                Bakery  0.11


----Acacias----
                venue  freq
0                 Bar  0.12
1         Art Gallery  0.07
2  Spanish Restaurant  0.07
3               Plaza  0.05
4         Pizza Place  0.05


----Adelfas----
                  venue  freq
0    Spanish Restaurant  0.08
1           Supermarket  0.05
2                   Bar  0.05
3        Breakfast Spot  0.05
4  Fast Food Restaurant  0.05


----Aeropuerto----
                     venue  freq
0         Business Service   1.0
1              Yoga Studio   0.0
2                     Park   0.0
3               Nail Salon   0.0
4  New American Restaurant   0.0


----Alame

                  venue  freq
0    Spanish Restaurant  0.11
1  Gym / Fitness Center  0.09
2                  Park  0.09
3                   Bar  0.09
4      Tapas Restaurant  0.06


----Fuentelarreina----
               venue  freq
0      Grocery Store   0.5
1              Hotel   0.5
2        Yoga Studio   0.0
3             Office   0.0
4  Paella Restaurant   0.0


----Gaztambide----
                venue  freq
0  Spanish Restaurant  0.10
1                Café  0.05
2    Tapas Restaurant  0.05
3                 Bar  0.05
4          Restaurant  0.05


----Goya----
                venue  freq
0  Spanish Restaurant  0.15
1          Restaurant  0.08
2    Tapas Restaurant  0.05
3                Café  0.04
4              Bakery  0.04


----Guindalera----
                venue  freq
0          Restaurant  0.12
1  Spanish Restaurant  0.09
2               Hotel  0.09
3       Grocery Store  0.06
4              Bakery  0.06


----Hellín----
         venue  freq
0         Park  0.25
1     Pharmac

              venue  freq
0  Tapas Restaurant  0.13
1              Café  0.08
2        Restaurant  0.05
3             Plaza  0.05
4               Bar  0.05


----San Juan Bautista----
                venue  freq
0  Spanish Restaurant  0.18
1                 Bar  0.11
2        Burger Joint  0.07
3          Restaurant  0.07
4              Bakery  0.04


----San Pascual----
                venue  freq
0       Grocery Store  0.17
1  Spanish Restaurant  0.13
2  Italian Restaurant  0.07
3               Hotel  0.07
4  Chinese Restaurant  0.03


----Santa Eugenia----
                venue  freq
0                 Gym  0.18
1  Spanish Restaurant  0.09
2        Soccer Field  0.09
3                 Bar  0.09
4      Farmers Market  0.09


----Simancas----
                venue  freq
0          Restaurant  0.15
1  Spanish Restaurant  0.15
2               Hotel  0.09
3      Sandwich Place  0.06
4                Café  0.06


----Sol----
                venue  freq
0               Hotel  0.09
1        

# put that into a pandas dataframe

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
# create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = madrid_grouped['Neighborhood']

for ind in np.arange(madrid_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(madrid_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Villaverde Alto,Train Station,Restaurant,Metro Station,Diner,Spanish Restaurant,Grocery Store,Mobile Phone Shop,Women's Store,Fast Food Restaurant,Fish & Chips Shop
1,Abrantes,Athletics & Sports,Plaza,Pizza Place,Soccer Field,Fast Food Restaurant,Bakery,Restaurant,Gym / Fitness Center,Metro Station,Women's Store
2,Acacias,Bar,Art Gallery,Spanish Restaurant,Plaza,Tapas Restaurant,Pizza Place,Gourmet Shop,Liquor Store,Bookstore,Gymnastics Gym
3,Adelfas,Spanish Restaurant,Bakery,Bar,Supermarket,Breakfast Spot,Tapas Restaurant,Fast Food Restaurant,Grocery Store,Boutique,Bookstore
4,Aeropuerto,Business Service,Women's Store,Farm,French Restaurant,Fountain,Football Stadium,Food Stand,Food Court,Food & Drink Shop,Food


## 4. Cluster Neighborhoods

In [39]:
# Run k-means to cluster the neighborhood into 5 clusters
# set number of clusters
kclusters = 5

madrid_grouped_clustering = madrid_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(madrid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 4, 3, 3, 3, 3, 3], dtype=int32)

In [40]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

madrid_merged = madrid_data

# merge madrid_grouped with madrid_data to add latitude/longitude for each neighborhood
madrid_merged = madrid_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

madrid_merged.head() # check the last columns!

,District,Number,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Centro,11,Palacio,28001,40.415129,-3.715618,3.0,Spanish Restaurant,Plaza,Tapas Restaurant,Historic Site,Restaurant,Bar,Vegetarian / Vegan Restaurant,Music Venue,Beer Bar,Café
1,Centro,12,Embajadores,28012,40.409681,-3.701644,3.0,Café,Plaza,Bar,Tapas Restaurant,Hostel,Spanish Restaurant,Coffee Shop,Theater,Art Gallery,Bookstore
2,Centro,13,Cortes,28014,40.414348,-3.698525,3.0,Restaurant,Plaza,Bar,Spanish Restaurant,Hotel,Tapas Restaurant,Mediterranean Restaurant,Café,Breakfast Spot,Bookstore
3,Centro,14,Justicia,28004,40.423957,-3.695747,3.0,Restaurant,Vegetarian / Vegan Restaurant,Bakery,Hotel,Spanish Restaurant,Italian Restaurant,Bookstore,Deli / Bodega,American Restaurant,Bistro
4,Centro,15,Universidad,28004,40.425310,-3.706630,3.0,Café,Plaza,Bookstore,Hotel,Tapas Restaurant,Bar,Theater,Japanese Restaurant,Restaurant,Cocktail Bar


#### Create a map of Madrid with neighborhoods superimposed on top.

In [70]:
# create map
print(latitude)
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(madrid_merged['Latitude'], madrid_merged['Longitude'], madrid_merged['Neighborhood'], madrid_merged['Cluster Labels']):
    if np.isnan(cluster):
        cluster = 0
    else:
        cluster = int(cluster)

    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

40.4167047


## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [64]:
madrid_merged.loc[madrid_merged['Cluster Labels'] == 0, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Number,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,82,-3.742602,0.0,Grocery Store,Hotel,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Women's Store
58,101,-3.743778,0.0,Grocery Store,Train Station,Toy / Game Store,Coffee Shop,Breakfast Spot,Metro Station,Food Court,Food & Drink Shop,Food,Flower Shop
68,114,-3.740044,0.0,Grocery Store,Fast Food Restaurant,Indie Theater,Tapas Restaurant,Metro Station,Stadium,Breakfast Spot,Spanish Restaurant,Coffee Shop,Toy / Game Store
72,121,-3.709874,0.0,Sporting Goods Shop,Farmers Market,Café,Accessories Store,Grocery Store,French Restaurant,Fountain,Football Stadium,Food Stand,Food Court
82,134,-3.634079,0.0,Café,Gas Station,Metro Station,Grocery Store,Flower Shop,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Women's Store
83,135,-3.648126,0.0,Bar,Grocery Store,Tapas Restaurant,Building,Dive Shop,Farmers Market,French Restaurant,Fountain,Football Stadium,Food Stand
85,141,-3.631709,0.0,Spanish Restaurant,Breakfast Spot,Athletics & Sports,Tea Room,Grocery Store,Flea Market,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market
108,173,-3.674240,0.0,Grocery Store,Women's Store,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Frozen Yogurt Shop
111,181,-3.621712,0.0,Grocery Store,Soccer Field,Basketball Court,Plaza,Church,Mobile Phone Shop,Food,Park,Fish & Chips Shop,Farmers Market


#### Cluster 2

In [65]:
madrid_merged.loc[madrid_merged['Cluster Labels'] == 1, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Number,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,65,-3.703423,1.0,Spanish Restaurant,Art Studio,Brewery,Mediterranean Restaurant,Soccer Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
43,81,-3.778074,1.0,Spanish Restaurant,Restaurant,Plaza,Government Building,Bakery,Tapas Restaurant,Fish Market,Farm,Farmers Market,Fast Food Restaurant
47,85,-3.695211,1.0,Spanish Restaurant,Italian Restaurant,College Gym,Park,Athletics & Sports,Food & Drink Shop,Restaurant,Food Court,Food,Flower Shop
54,94,-3.717074,1.0,Spanish Restaurant,Supermarket,Soccer Field,Asian Restaurant,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market
56,96,-3.828783,1.0,Paella Restaurant,Beer Garden,Shop & Service,Farm,Spanish Restaurant,Food Stand,Food Court,Food & Drink Shop,Food,Falafel Restaurant
77,126,-3.715285,1.0,Spanish Restaurant,Park,Beer Garden,Athletics & Sports,Farm,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
78,127,-3.707778,1.0,Spanish Restaurant,Athletics & Sports,Beer Garden,Park,Theater,Food Stand,Food Court,Food & Drink Shop,Food,Farm
79,131,-3.674761,1.0,Park,Gym / Fitness Center,Turkish Restaurant,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market
81,133,-3.657983,1.0,Toy / Game Store,Park,Food & Drink Shop,Bar,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market
86,142,-3.626776,1.0,Spanish Restaurant,Pizza Place,Bakery,Soccer Stadium,Italian Restaurant,Soccer Field,Park,Flea Market,Farmers Market,Fast Food Restaurant


#### Cluster 3

In [66]:
madrid_merged.loc[madrid_merged['Cluster Labels'] == 2, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Number,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,91,-3.755858,2.0,Trail,Women's Store,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Falafel Restaurant


#### Cluster 4

In [67]:
madrid_merged.loc[madrid_merged['Cluster Labels'] == 3, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Number,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,11,-3.715618,3.0,Spanish Restaurant,Plaza,Tapas Restaurant,Historic Site,Restaurant,Bar,Vegetarian / Vegan Restaurant,Music Venue,Beer Bar,Café
1,12,-3.701644,3.0,Café,Plaza,Bar,Tapas Restaurant,Hostel,Spanish Restaurant,Coffee Shop,Theater,Art Gallery,Bookstore
2,13,-3.698525,3.0,Restaurant,Plaza,Bar,Spanish Restaurant,Hotel,Tapas Restaurant,Mediterranean Restaurant,Café,Breakfast Spot,Bookstore
3,14,-3.695747,3.0,Restaurant,Vegetarian / Vegan Restaurant,Bakery,Hotel,Spanish Restaurant,Italian Restaurant,Bookstore,Deli / Bodega,American Restaurant,Bistro
4,15,-3.706630,3.0,Café,Plaza,Bookstore,Hotel,Tapas Restaurant,Bar,Theater,Japanese Restaurant,Restaurant,Cocktail Bar
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,206,-3.569656,3.0,Restaurant,Café,Spanish Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Women's Store
124,207,-3.608456,3.0,Restaurant,Hotel,Pizza Place,Breakfast Spot,Supermarket,Café,Smoke Shop,Asian Restaurant,Spanish Restaurant,Plaza
125,208,-3.627987,3.0,Mediterranean Restaurant,Italian Restaurant,Spanish Restaurant,Restaurant,Hotel,Asian Restaurant,Supermarket,Park,Cafeteria,Breakfast Spot
126,211,-3.587975,3.0,Tapas Restaurant,Restaurant,Hotel,Bakery,Shop & Service,Cocktail Bar,Bookstore,Scenic Lookout,Metro Station,Bar


#### Cluster 5

In [68]:
madrid_merged.loc[madrid_merged['Cluster Labels'] == 4, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Number,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
127,212,-3.574081,4.0,Business Service,Women's Store,Farm,French Restaurant,Fountain,Football Stadium,Food Stand,Food Court,Food & Drink Shop,Food
